In [54]:
from langchain.document_loaders import TextLoader, UnstructuredURLLoader,UnstructuredCSVLoader

In [55]:
url_loader=UnstructuredURLLoader(urls=[
    "https://edition.cnn.com/markets/stocks/TSLA"
    
])

In [56]:
data=url_loader.load()
len(data)

1

In [57]:
from langchain.text_splitter import RecursiveCharacterTextSplitter  

splitter=RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100
)

In [58]:
docs=splitter.split_documents(data)

In [59]:
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.chains.qa_with_sources.loading import load_qa_with_sources_chain
from langchain.vectorstores import FAISS  
from sentence_transformers import SentenceTransformer
from langchain.embeddings import HuggingFaceEmbeddings
import pickle



In [60]:

embedding_model = HuggingFaceEmbeddings(model_name="all-mpnet-base-v2")
index = FAISS.from_documents(docs, embedding_model)
file_path = "FAISS_Vector_Index.pkl"
with open(file_path, "wb") as f:
    pickle.dump(index, f)


In [61]:
with open(file_path,"rb") as f:
    vector_ind=pickle.load(f)

In [62]:
from langchain_ollama.llms import OllamaLLM


In [63]:
llm = OllamaLLM(model="llama3.1")

chain=RetrievalQAWithSourcesChain.from_llm(llm=llm,retriever=vector_ind.as_retriever())

In [64]:
chain

RetrievalQAWithSourcesChain(verbose=False, combine_documents_chain=MapReduceDocumentsChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Use the following portion of a long document to see if any of the text is relevant to answer the question. \nReturn any relevant text verbatim.\n{context}\nQuestion: {question}\nRelevant text, if any:'), llm=OllamaLLM(model='llama3.1'), output_parser=StrOutputParser(), llm_kwargs={}), reduce_documents_chain=ReduceDocumentsChain(verbose=False, combine_documents_chain=StuffDocumentsChain(verbose=False, llm_chain=LLMChain(verbose=False, prompt=PromptTemplate(input_variables=['question', 'summaries'], input_types={}, partial_variables={}, template='Given the following extracted parts of a long document and a question, create a final answer with references ("SOURCES"). \nIf you don\'t know the answer, just say that you don\'t know. Don\'t try t

In [65]:
import langchain

In [66]:
question="What is Tesla Market Cap?"
langchain.debug=True

chain({"question":question},return_only_outputs=True)

[chain/start] [chain:RetrievalQAWithSourcesChain] Entering Chain run with input:
{
  "question": "What is Tesla Market Cap?"
}
[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQAWithSourcesChain > chain:MapReduceDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "input_list": [
    {
      "context": "Thank You!\n\nYour effort and contribution in providing this feedback is much appreciated.\n\nClose\n\nAd Feedback\n\nTSLA\n\nTesla, Inc.\n\nTesla, Inc.\n\nTSLA\n\nKey terms\n\nAbout TSLA\n\nTesla, Inc. engages in the design, development, manufacture, and sale of electric vehicles and energy generation and storage systems. It operates through the Automotive and Energy Generation and Storage segments. The Automotive segment includes the design, development, manufacture, sale, and lease of electric vehicles as well as sales of automotive regulatory credits. The Energy Gene

{'answer': "I cannot provide the final answer yet.\n\nThe provided document does not mention Tesla's Market Cap. \n\nHowever, I can tell you that according to CNN (https://edition.cnn.com/markets/stocks/TSLA), Tesla's market capitalization is a publicly available figure and can be looked up on various financial websites such as Yahoo Finance or Google Finance.",
 'sources': ''}